<a href="https://colab.research.google.com/github/BaoFrancisNguyen/IA/blob/main/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Traitement et manipulation des données
import pandas as pd
import numpy as np

    # Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

    # Modélisation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
import pandas as pd

#Lien direct de téléchargement
url = "https://drive.google.com/uc?export=download&id=1TzUbV-Ad0LlZ-oXvlRoT0gUPKtb0UP7X"

    # Charger les données
df = pd.read_csv(url, sep="\t")

    # Aperçu des données\n",
print(df.head())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...                  7             0   
1  08-03-2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   
3  10-02-2014       26        11  ...                  6             0   
4  19-01-2014       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  \
0             0

In [5]:
colonnes_vides = df.isnull().sum()
print(colonnes_vides[colonnes_vides > 0])  # Afficher uniquement les colonnes avec des valeurs manquantes


Income    24
dtype: int64


In [6]:

# pour remplacer les éléments vides par la moyenne
#moyenne_income = df['Income'].mean()
     #df['Income'] = df['Income'].fillna(moyenne_income)

# pour supprimer les lignes n'ayant pas de données dans la colonne "Income"
df = df.dropna(subset=['Income'])


In [7]:
#identifier les doublons
doublons = df.duplicated()
print(doublons.sum())
 #si doublons, utiliser df.drop_duplicates()

0


In [8]:
# supprimer la colonne 'ID'
df = df.drop('ID', axis=1)
print(df)

      Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0           1957  Graduation         Single  58138.0        0         0   
1           1954  Graduation         Single  46344.0        1         1   
2           1965  Graduation       Together  71613.0        0         0   
3           1984  Graduation       Together  26646.0        1         0   
4           1981         PhD        Married  58293.0        1         0   
...          ...         ...            ...      ...      ...       ...   
2235        1967  Graduation        Married  61223.0        0         1   
2236        1946         PhD       Together  64014.0        2         1   
2237        1981  Graduation       Divorced  56981.0        0         0   
2238        1956      Master       Together  69245.0        0         1   
2239        1954         PhD        Married  52869.0        1         1   

     Dt_Customer  Recency  MntWines  MntFruits  ...  NumWebVisitsMonth  \
0     04-09-2012       58

In [9]:
valeurs_uniques = df['Marital_Status'].unique()
print(valeurs_uniques)


['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']


In [10]:
comptage_valeurs = df['Marital_Status'].value_counts()
print(comptage_valeurs)


Marital_Status
Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64


In [11]:
# Supprimer toutes les occurrences de 'Absurd' et 'Yolo'dans la colonne 'Marital_Status'
df = df[df['Marital_Status'] != 'Absurd']
df = df[df['Marital_Status'] != 'YOLO']


print(df['Marital_Status'].value_counts())


Marital_Status
Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Name: count, dtype: int64


Encodage des données de **catégories**

In [28]:
from sklearn.preprocessing import LabelEncoder
# Initialiser le label encoder
label_encoder = LabelEncoder()

# Liste des colonnes catégoriques à encoder
categorical_columns = ['Education', 'Marital_Status']

# pour les colonnes 'Education' et 'Marital_Status'
##label_encoder = LabelEncoder()
##data['Education_Label'] = label_encoder.fit_transform(data['Education'])
##data['Marital_Status_Label'] = label_encoder.fit_transform(data['Marital_Status'])

# Appliquer à plusieurs colonnes
categorical_columns = ['Education', 'Marital_Status']
for col in categorical_columns:
    df[col + '_Label'] = label_encoder.fit_transform(df[col])

# Vérifier le résultat
print(df[['Education', 'Education_Label']])


       Education  Education_Label
0     Graduation                2
1     Graduation                2
2     Graduation                2
3     Graduation                2
4            PhD                4
...          ...              ...
2235  Graduation                2
2236         PhD                4
2237  Graduation                2
2238      Master                3
2239         PhD                4

[2212 rows x 2 columns]


**Nous avons défini les données d'Education' et 'Marital_Status' en int**


In [13]:
valeurs_uniques_education = df['Education_Label'].unique()
print(valeurs_uniques_education) # vérifier toutes les valeurs différentes

[2 4 3 1 0]


In [38]:
print(df.isnull().sum())  # vérifier si on compte des NaN

Year_Birth              0
Education               0
Marital_Status          0
Income                  0
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
Education_Label         0
Marital_Status_Label    0
dtype: int64


In [39]:
print(df)

      Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0           1957  Graduation          Alone  58138.0        0         0   
1           1954  Graduation          Alone  46344.0        1         1   
2           1965  Graduation       Together  71613.0        0         0   
3           1984  Graduation       Together  26646.0        1         0   
4           1981         PhD       Together  58293.0        1         0   
...          ...         ...            ...      ...      ...       ...   
2235        1967  Graduation       Together  61223.0        0         1   
2236        1946         PhD       Together  64014.0        2         1   
2237        1981  Graduation          Alone  56981.0        0         0   
2238        1956      Master       Together  69245.0        0         1   
2239        1954         PhD       Together  52869.0        1         1   

     Dt_Customer  Recency  MntWines  MntFruits  ...  AcceptedCmp4  \
0     04-09-2012       58     

In [16]:

df['Marital_Status'] = df['Marital_Status'].replace('Married', 'Together')
df['Marital_Status'] = df['Marital_Status'].replace('Divorced', 'Alone')
df['Marital_Status'] = df['Marital_Status'].replace('Widow', 'Alone')
df['Marital_Status'] = df['Marital_Status'].replace('Single', 'Alone')
print(df['Marital_Status'].value_counts())

Marital_Status
Together    1430
Alone        782
Name: count, dtype: int64


In [32]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Exemple de DataFrame avec les colonnes nécessaires


# Sélection des colonnes numériques
numerical_cols = ['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
                  'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
                  'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                  'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
                  'AcceptedCmp2', 'Z_CostContact', 'Z_Revenue','Response']

# Sélection des colonnes catégoriques
categorical_cols = ['Dt_Customer', 'Complain']

# Sélectionner les colonnes numériques pour la standardisation
X = df[numerical_cols]

# Créer un objet StandardScaler
scaler = StandardScaler()

# Appliquer la standardisation
X_scaled = scaler.fit_transform(X)

# Convertir les données standardisées en DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=numerical_cols)

# Concaténer les colonnes standardisées avec les variables catégoriques
df_standardized = pd.concat([X_scaled_df, df[categorical_cols]], axis=1)

# Vérification du résultat
print(df_standardized.head())



     Income   Kidhome  Teenhome   Recency  MntWines  MntFruits  \
0  0.234515 -0.824288  -0.92886  0.309300  0.977586   1.550693   
1 -0.233840  1.038148   0.90892 -0.381922 -0.871112  -0.636731   
2  0.769624 -0.824288  -0.92886 -0.796656  0.358390   0.570123   
3 -1.016073  1.038148  -0.92886 -0.796656 -0.871112  -0.561303   
4  0.240670  1.038148  -0.92886  1.553500 -0.391162   0.419267   

   MntMeatProducts  MntFishProducts  MntSweetProducts  MntGoldProds  ...  \
0         1.689878         2.465495          1.483638      0.855961  ...   
1        -0.717674        -0.651121         -0.634018     -0.732823  ...   
2        -0.178204         1.347180         -0.147201     -0.035308  ...   
3        -0.655256        -0.504457         -0.585336     -0.752199  ...   
4        -0.218330         0.155532         -0.001155     -0.558445  ...   

   AcceptedCmp3  AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  \
0     -0.282048     -0.282981     -0.280175     -0.260927     -0.117256

In [33]:

df_standardized['Education_Label']
valeurs_uniques_education_standardized = df_standardized['Education_Label'].unique()
print(valeurs_uniques_education_standardized)

KeyError: 'Education_Label'

In [40]:
import pandas as pd

# Conversion en datetime
df_standardized["Dt_Customer"] = pd.to_datetime(df_standardized["Dt_Customer"], format="%d-%m-%Y")

# Extraire l'année et le mois (ou tout autre paramètre pertinent)
df_standardized["Year_Customer"] = df_standardized["Dt_Customer"].dt.year
df_standardized["Month_Customer"] = df_standardized["Dt_Customer"].dt.month

# Option : Nombre de jours écoulés depuis une date de référence
df_standardized["Days_Since_Customer"] = (pd.to_datetime("2024-01-01") - df_standardized["Dt_Customer"]).dt.days


In [41]:
from sklearn.preprocessing import StandardScaler

# Liste des colonnes numériques à standardiser
num_cols = ["Income", "Kidhome", "Teenhome", "Recency", "MntWines", "MntFruits",
            "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds",
            "Year_Customer", "Month_Customer", "Days_Since_Customer"]

scaler = StandardScaler()
df_standardized[num_cols] = scaler.fit_transform(df_standardized[num_cols])


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Séparer les caractéristiques et la cible
X = df_standardized.drop(columns=["Response", "Dt_Customer"])  # Supprimer aussi Dt_Customer transformé
y = df_standardized["Response"]

# Handle missing values in the target variable (y) using SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')  # Replace NaNs with the most frequent value
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))  # Reshape for SimpleImputer compatibility
y_imputed = y_imputed.astype(int)

y = y_imputed.ravel()

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialisation et entraînement
model = RandomForestClassifier(random_state=10)
model.fit(X_train, y_train)

# Prédiction et évaluation
y_pred = model.predict(X_test)
print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy : 0.859375
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       382
           2       0.58      0.17      0.26        66

    accuracy                           0.86       448
   macro avg       0.73      0.57      0.59       448
weighted avg       0.83      0.86      0.82       448



In [27]:
#identifier les types
print(X_train.dtypes)


Income                  float64
Kidhome                 float64
Teenhome                float64
Recency                 float64
MntWines                float64
MntFruits               float64
MntMeatProducts         float64
MntFishProducts         float64
MntSweetProducts        float64
MntGoldProds            float64
NumDealsPurchases       float64
NumWebPurchases         float64
NumCatalogPurchases     float64
NumStorePurchases       float64
NumWebVisitsMonth       float64
AcceptedCmp3            float64
AcceptedCmp4            float64
AcceptedCmp5            float64
AcceptedCmp1            float64
AcceptedCmp2            float64
Z_CostContact           float64
Z_Revenue               float64
Education_Label         float64
Marital_Status_Label    float64
Complain                float64
dtype: object


In [ ]:

X_train = pd.get_dummies(X_train, columns=["Education", "Marital_Status"], drop_first=True)
X_test = pd.get_dummies(X_test, columns=["Education", "Marital_Status"], drop_first=True)


KeyError: "None of [Index(['Education', 'Marital_Status'], dtype='object')] are in the [columns]"

In [ ]:
print(X_train.dtypes)

ID                           int64
Year_Birth                   int64
Income                     float64
Kidhome                    float64
Teenhome                   float64
Recency                    float64
MntWines                   float64
MntFruits                  float64
MntMeatProducts            float64
MntFishProducts            float64
MntSweetProducts           float64
MntGoldProds               float64
NumDealsPurchases            int64
NumWebPurchases              int64
NumCatalogPurchases          int64
NumStorePurchases            int64
NumWebVisitsMonth            int64
AcceptedCmp3                 int64
AcceptedCmp4                 int64
AcceptedCmp5                 int64
AcceptedCmp1                 int64
AcceptedCmp2                 int64
Complain                     int64
Z_CostContact                int64
Z_Revenue                    int64
Year_Customer              float64
Month_Customer             float64
Days_Since_Customer        float64
Education_Basic     

In [ ]:
# S'assurer que les colonnes correspondent entre X_train et X_test
Si vous utilisez One-Hot Encoding, il peut arriver que X_train et X_test aient des colonnes différentes. Harmonisez-les comme suit :
X_train, X_test = X_train.align(X_test, join="left", axis=1)
X_test = X_test.fillna(0)  # Remplace les colonnes manquantes par 0


In [ ]:
model = RandomForestClassifier(random_state=11)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy : 0.8727678571428571
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       379
           1       0.71      0.29      0.41        69

    accuracy                           0.87       448
   macro avg       0.80      0.63      0.67       448
weighted avg       0.86      0.87      0.85       448



In [ ]:
print(y_train.value_counts())


Response
0    1527
1     265
Name: count, dtype: int64


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = undersample.fit_resample(X_train, y_train)



In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy : 0.8660714285714286
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       379
           1       0.67      0.26      0.38        69

    accuracy                           0.87       448
   macro avg       0.77      0.62      0.65       448
weighted avg       0.85      0.87      0.84       448



In [ ]:
# Prédiction avec un seuil ajusté (par exemple, 0.3 au lieu de 0.5)
y_pred = (model.predict_proba(X_test)[:, 1] > 0.3).astype(int)


In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = (model.predict_proba(X_test)[:, 1] > 0.3677).astype(int)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy : 0.8571428571428571
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       379
           1       0.55      0.41      0.47        69

    accuracy                           0.86       448
   macro avg       0.72      0.67      0.69       448
weighted avg       0.84      0.86      0.85       448



In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Simuler des données pour l'exemple
data = {
    'Income': np.random.rand(100),
    'Kidhome': np.random.randint(0, 2, 100),
    'Teenhome': np.random.randint(0, 2, 100),
    'Recency': np.random.rand(100),
    'MntWines': np.random.rand(100),
    'MntFruits': np.random.rand(100),
    'Response': np.random.randint(0, 2, 100),
}
df = pd.DataFrame(data)

# Séparer les features (X) et la cible (y)
X = df.drop('Response', axis=1)
y = df['Response']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraînement du modèle
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Prédictions
y_pred = model.predict(X_test_scaled)

# Obtenir les métriques de classification
report = classification_report(y_test, y_pred, output_dict=True)

# Créer le dashboard
st.title("Dashboard de Performance du Modèle")

# Affichage des résultats
st.subheader("Métriques de classification")
st.write(pd.DataFrame(report).transpose())

# Graphiques
st.subheader("Distribution des Prédictions")
st.bar_chart(pd.Series(y_pred).value_counts())

st.subheader("Distribution des Labels Réels")
st.bar_chart(pd.Series(y_test).value_counts())


2024-12-02 15:03:11.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-02 15:03:11.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [ ]:
streamlit run votre_script.py
